In [295]:
import geopandas
import pandas as pd
import geopandas as gpd
import folium

In [296]:
data = pd.read_csv('data/poland/RYNE_3786_CTAB_20250509085604.csv', sep=';')

In [297]:
data.head()

,Code,Name,total;total;2020;[PLN],total;total;2021;[PLN],total;total;2022;[PLN],total;total;2023;[PLN],total;total;2024;[PLN],total;to 40 m2;2020;[PLN],total;to 40 m2;2021;[PLN],total;to 40 m2;2022;[PLN],...,secondary market;from 60.1 to 80 m2;2021;[PLN],secondary market;from 60.1 to 80 m2;2022;[PLN],secondary market;from 60.1 to 80 m2;2023;[PLN],secondary market;from 60.1 to 80 m2;2024;[PLN],secondary market;from 80.1 m2;2020;[PLN],secondary market;from 80.1 m2;2021;[PLN],secondary market;from 80.1 m2;2022;[PLN],secondary market;from 80.1 m2;2023;[PLN],secondary market;from 80.1 m2;2024;[PLN],Unnamed: 77
0,0,POLAND,311467,348988,393524,427466,NaN,201152,226284,261041,...,388297,452500,512719,NaN,549670,607904,758603,829766,NaN,NaN
1,200000,DOLNOŚLĄSKIE,268487,340342,380369,419625,NaN,167974,222302,258875,...,352027,404170,463270,NaN,390941,467390,540474,616124,NaN,NaN
2,201000,Powiat bolesławiecki,223395,225740,248481,287441,NaN,116331,127699,168995,...,269723,289078,394540,NaN,304779,320632,345199,420925,NaN,NaN
3,202000,Powiat dzierżoniowski,115134,146681,156434,223186,NaN,73698,87044,102323,...,187085,202571,247649,NaN,202386,311299,236726,359145,NaN,NaN
4,203000,Powiat głogowski,215504,235327,263513,278506,NaN,132053,152369,155974,...,289564,303585,313409,NaN,352675,400239,431744,489586,NaN,NaN


In [298]:
new_df = data.iloc[:, [0, 1, 5]].rename(columns={"total;total;2023;[PLN]": "Price"})
new_df["Price Increase"] = (new_df["Price"]-data.iloc[:,2])/data.iloc[:,2]
new_df.head()

,Code,Name,Price,Price Increase
0,0,POLAND,427466,0.372428
1,200000,DOLNOŚLĄSKIE,419625,0.562925
2,201000,Powiat bolesławiecki,287441,0.286694
3,202000,Powiat dzierżoniowski,223186,0.938489
4,203000,Powiat głogowski,278506,0.292347


In [299]:
mapa_pow = gpd.read_file('data/poland/powiaty/powiaty.shp')
# ❗ Fix: reproject to EPSG:4326
mapa_pow = mapa_pow.to_crs(epsg=4326)
mapa_pow = mapa_pow[['JPT_KOD_JE', 'geometry']]

In [300]:
average_m2_gus = new_df.copy()

average_m2_gus['Code (Filtered)'] = average_m2_gus.Code.apply(lambda x: '0'+str(x) if len(str(x)) < 7 else str(x))
average_m2_gus['Code (Filtered)'] = average_m2_gus['Code (Filtered)'].apply(lambda s: s[:4])
average_m2_gus = average_m2_gus[average_m2_gus['Code'] != '0']
data_gus_pow = average_m2_gus[average_m2_gus['Code (Filtered)'].str[0:3] != '000']


In [301]:
data_gus_2 = data_gus_pow[data_gus_pow['Code (Filtered)'].str[0:2] != '00']
data_gus_2 = data_gus_2[data_gus_2['Code (Filtered)'].str[2:4] != '00']

In [302]:
data_gus_2['Price'] = pd.to_numeric(data_gus_2['Price'], errors='coerce')

In [303]:
mapa_pow['geometry'] = mapa_pow['geometry'].buffer(0)

In [304]:
mapa_pow.geometry = mapa_pow.geometry.simplify(0.001)
#mapa_pow['geometry'] = mapa_pow['geometry'].simplify(tolerance=0.001, preserve_topology=True)
pow_geoPath = mapa_pow.to_json()
mapa = folium.Map([52, 19], zoom_start=6)

folium.Choropleth(geo_data=pow_geoPath,  # GeoJSON
                  data=data_gus_2,
                  columns=['Code (Filtered)', 'Price'],
                  key_on='feature.properties.JPT_KOD_JE',  # key to GeoJSON
                  fill_color='YlOrRd',
                  fill_opacity=0.7,
                  line_opacity=0.2,
                  legend_name="Median Transaction Price").add_to(mapa)

In [305]:
m2 =mapa_pow.merge(data_gus_2[['Code (Filtered)', 'Price']], left_on='JPT_KOD_JE', right_on='Code (Filtered)', how='left')
m2["Price(EUR)"] = m2["Price"] / 4.22
folium.GeoJson(
    data=m2.to_json(),
    name='Price Tooltip',
    tooltip=folium.features.GeoJsonTooltip(
        fields=['Price', 'Price(EUR)'],
        aliases=['Average Price (PLN):', 'Average Price (EUR):'],
        localize=True
    ),
    style_function=lambda x: {
        'fillColor': 'transparent',
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.1,
    }
).add_to(mapa)

In [306]:
mapa.save(outfile="exports/map.html")

In [307]:
import numpy as np
data_gus_3 = data_gus_2.copy().replace([np.inf, -np.inf], np.nan)

In [308]:
mapa2 = folium.Map([52, 19], zoom_start=6)

folium.Choropleth(geo_data=pow_geoPath,  # GeoJSON
                  data=data_gus_3,
                  columns=['Code (Filtered)', 'Price Increase'],
                  key_on='feature.properties.JPT_KOD_JE',  # key to GeoJSON
                  fill_color='YlOrRd',
                  fill_opacity=0.7,
                  line_opacity=0.2,
                  legend_name="Median Prince Increase").add_to(mapa2)

In [309]:
data_gus_3

,Code,Name,Price,Price Increase,Code (Filtered)
2,201000,Powiat bolesławiecki,287441,0.286694,0201
3,202000,Powiat dzierżoniowski,223186,0.938489,0202
4,203000,Powiat głogowski,278506,0.292347,0203
5,204000,Powiat górowski,146458,0.408318,0204
6,205000,Powiat jaworski,281036,0.578304,0205
...,...,...,...,...,...
392,3217000,Powiat wałecki,233386,0.278561,3217
393,3218000,Powiat łobeski,200607,0.336186,3218
394,3261000,City with powiat status Koszalin,329357,NaN,3261
395,3262000,City with powiat status Szczecin,487867,0.223893,3262


In [310]:
m3 =mapa_pow.merge(data_gus_3, left_on='JPT_KOD_JE', right_on='Code (Filtered)', how='left')
#m3["Price(EUR)"] = m3["Price Increase"] / 4.22
m3["Price Increase"] = (m3["Price Increase"]*100).round(2)
folium.GeoJson(
    data=m3.to_json(),
    name='Price Tooltip',
    tooltip=folium.features.GeoJsonTooltip(
        fields=["Name", 'Price Increase'],#, 'Price(EUR)'],
        aliases=["Powiat Name",'Median Price Increase (PLN):'],#, 'Median Price Increase (EUR):'],
        localize=True
    ),
    style_function=lambda x: {
        'fillColor': 'transparent',
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.1,
    }
).add_to(mapa2)

In [311]:
mapa2.save(outfile="exports/map2.html")